In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os,shutil
from os.path import isfile, join
%matplotlib inline


In [29]:
# lst=['bike']
#clearing answer folder
my_Folder_Name = 'answer' #This is a string that I generate
if not os.path.exists(my_Folder_Name):
    os.makedirs(my_Folder_Name)
folder=my_Folder_Name+'/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
        
t=0
for x in range(0,112):
    count=0
    t = x
    while t>0:
        t=int(t/10)
        count+=1
    num=""
    for i in range(0,6-count):
        num = num + str(0)
    num = num + str(x)
    
    if x is 0 :
        num="000000"
    imgL=Image.open('images/I1_'+num+'.png')
    imgL=np.array(imgL)
    imgR=Image.open('images/I2_'+num+'.png')
    imgR=np.array(imgR)
    disparityrange = 160
    try:
        zero = np.zeros((imgL.shape[0],disparityrange))
    except:
        break
    imgL = np.hstack((zero,imgL))
    imgR = np.hstack((zero,imgR))
    
    imgL = imgL.astype(np.uint8)
    imgR = imgR.astype(np.uint8)
    
#     plt.figure(figsize=(10,10))
#     plt.imshow(imgL)
#     plt.title("Left")
#     plt.show()
    
#     plt.figure(figsize=(10,10))
#     plt.imshow(imgR)
#     plt.title("Right")
#     plt.show()
    
    window_size = 3  # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
    
    left_matcher = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=disparityrange,  # max_disp has to be dividable by 16 f. E. HH 192, 256
        blockSize=5,
        P1=8 * 3 * window_size ** 2,
        # wsize default 3; 5; 7 for SGBM reduced size image; 15 for SGBM full size image (1300px and above); 5 Works nicely
        P2=32 * 3 * window_size ** 2,
        disp12MaxDiff=-1,
        uniquenessRatio=15,
        speckleWindowSize=0,
        speckleRange=2,
        preFilterCap=63,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )
    right_matcher = cv2.ximgproc.createRightMatcher(left_matcher)
    
    print('computing disparity...',num)
    
    displ = left_matcher.compute(imgL, imgR)#.astype(np.float32)/16
    dispr = right_matcher.compute(imgR, imgL)#.astype(np.float32)/16
    displ = np.int16(displ)
    dispr = np.int16(dispr)
    #wls is used for smoothing images which are holefree
    
    #Disparity map filter based on Weighted Least Squares filter (in form of Fast Global Smoother that
    #is a lot faster than traditional Weighted Least Squares filter implementations)
    #and optional use of left-right-consistency-based confidence to refine the results in half-occlusions and uniform areas.
    lmbda = 80000
    sigma = 1.2
    visual_multiplier = 1

    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=left_matcher)
    wls_filter.setLambda(lmbda)
    wls_filter.setSigmaColor(sigma)

    
    filteredImg = wls_filter.filter(displ, imgL, None, dispr)  # important to put "imgL" here!!!
    #When the normType is NORM_MINMAX, cv::normalize normalizes _src in such a way that
    #the min value of dst is alpha and max value of dst is beta. 
    #cv::normalize does its magic using only scales and shifts (i.e. adding constants and multiplying by constants).
    #dst = (x-a)/(b-a)*(beta-alpha) + alpha    a & b are lower and upper bound of normalization
    filteredImg = cv2.normalize(src=filteredImg, dst=filteredImg, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
    filteredImg = np.uint8(filteredImg)
    filteredImg=np.delete(filteredImg, np.s_[0:disparityrange], axis=1)
#     plt.figure(figsize=(20,10))
#     plt.imshow(filteredImg)
#     plt.title(num)
#     plt.show()
    path = 'answer/'
    # saving depth images in answer folder
    path = path + str(x) + '.jpg'
    cv2.imwrite(path, filteredImg)
    print(str(x) + ' done')
    t=t+1

pathIn= 'answer/'
pathOut = 'video.avi'
fps = 25.0

frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
#for sorting the file names properly
files.sort(key = lambda x: int(x[0:-4]))
 
for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    print(filename)
    #inserting the frames into an image array
    frame_array.append(img)

out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()
   

FileNotFoundError: [Errno 2] No such file or directory: 'images/I1_000000.png'

answer/0.jpg
answer/1.jpg
answer/2.jpg
answer/3.jpg
answer/4.jpg
answer/5.jpg
answer/6.jpg
answer/7.jpg
answer/8.jpg
answer/9.jpg
answer/10.jpg
answer/11.jpg
answer/12.jpg
answer/13.jpg
answer/14.jpg
answer/15.jpg
answer/16.jpg
answer/17.jpg
answer/18.jpg
answer/19.jpg
answer/20.jpg
answer/21.jpg
answer/22.jpg
answer/23.jpg
answer/24.jpg
answer/25.jpg
answer/26.jpg
answer/27.jpg
answer/28.jpg
answer/29.jpg
answer/30.jpg
answer/31.jpg
answer/32.jpg
answer/33.jpg
answer/34.jpg
answer/35.jpg
answer/36.jpg
answer/37.jpg
answer/38.jpg
answer/39.jpg
answer/40.jpg
answer/41.jpg
answer/42.jpg
answer/43.jpg
answer/44.jpg
answer/45.jpg
answer/46.jpg
answer/47.jpg
answer/48.jpg
answer/49.jpg
answer/50.jpg
answer/51.jpg
answer/52.jpg
answer/53.jpg
answer/54.jpg
answer/55.jpg
answer/56.jpg
answer/57.jpg
answer/58.jpg
answer/59.jpg
answer/60.jpg
answer/61.jpg
answer/62.jpg
answer/63.jpg
answer/64.jpg
answer/65.jpg
answer/66.jpg
answer/67.jpg
answer/68.jpg
answer/69.jpg
answer/70.jpg
answer/71.jpg
an

In [ ]:
https://github.com/opencv/opencv/blob/master/modules/calib3d/src/stereosgbm.cpp

2258
2133
67

https://github.com/opencv/opencv_contrib/blob/master/modules/ximgproc/src/disparity_filters.cpp 453

2152
1.struct StereoSGBMParams
2.static void calcPixelCostBT( const Mat& img1, const Mat& img2, int y,
                            int minD, int maxD, CostType* cost,
                            PixType* buffer, const PixType* tab,
                            int tabOfs, int , int xrange_min = 0, int xrange_max = DEFAULT_RIGHT_BORDER )
3.static void computeDisparitySGBM( const Mat& img1, const Mat& img2,
                                 Mat& disp1, const StereoSGBMParams& params,
                                 Mat& buffer )

4.static void computeDisparitySGBM_HH4( const Mat& img1, const Mat& img2,
                                 Mat& disp1, const StereoSGBMParams& params,
                                 Mat& buffer )
5.void getBufferPointers(Mat& buffer, int width, int width1, int D, int num_ch, int SH2, int P2,
                       CostType*& curCostVolumeLine, CostType*& hsumBuf, CostType*& pixDiff,
                       PixType*& tmpBuf, CostType*& horPassCostVolume,
                       CostType*& vertPassCostVolume, CostType*& vertPassMin, CostType*& rightPassBuf,
                       CostType*& disp2CostBuf, short*& disp2Buf);
6.void SGBM3WayMainLoop::getRawMatchingCost(CostType* C, // target cost-volume row
                                          CostType* hsumBuf, CostType* pixDiff, PixType* tmpBuf, //buffers
                                          int y, int src_start_idx) const
7.inline void accumulateCostsLeftTop(CostType* leftBuf, CostType* leftBuf_prev, CostType* topBuf, CostType* costs,
                                   CostType& leftMinCost, CostType& topMinCost, int D, int P1, int P2)
8.inline void accumulateCostsRight(CostType* rightBuf, CostType* topBuf, CostType* leftBuf, CostType* costs,
                                 CostType& rightMinCost, int D, int P1, int P2, int& optimal_disp, CostType& min_cost)

9.void SGBM3WayMainLoop::operator () (const Range& range) const
10.static void computeDisparity3WaySGBM( const Mat& img1, const Mat& img2,
                                      Mat& disp1, const StereoSGBMParams& params,
                                      Mat* buffers, int nstripes )
11.class StereoSGBMImpl CV_FINAL : public StereoSGBM
    